In [1]:
def add_numbers(a: int, b: int) -> int:
    return a + b

In [2]:
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import os

add_numbers_tool = Tool(
    name="AddNumbers",
    func=lambda query: add_numbers(*map(int, query.split())),
    description="Adds two numbers together. Input should be two integers separated by a space."
)

llm = OpenAI(temperature=0, streaming=True)

tools = [add_numbers_tool]
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

prompt = "3 더하기 4는 얼마?"
response = agent.run(prompt)

print(response)


/tmp/ipykernel_21732/1299944318.py:12: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0, streaming=True)
/tmp/ipykernel_21732/1299944318.py:15: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGr



> Entering new AgentExecutor chain...
 I should use the AddNumbers function to add 3 and 4
Action: AddNumbers
Action Input: 3 4
Observation: 7
Thought: I now know the final answer
Final Answer: 7

> Finished chain.
7


In [3]:
import json

def get_current_weather(location, unit="섭씨"):
    weather_info = {
        "장소": location,
        "온도": "24",
        "단위": unit,
        "예보": ["비"],
    }
    return json.dumps(weather_info, ensure_ascii=False)

In [4]:
get_current_weather_tool = Tool(
    name="GetCurrentWeather",
    func=lambda query: get_current_weather(*query.split(',')),
    description="Returns the current weather for a given location. Input should be the location followed by the unit (optional) separated by a comma. Answer in Korean"
)


In [5]:
llm = OpenAI(temperature=0)

tools = [get_current_weather_tool]
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

prompt = "서울의 날씨를 알려줘."
response = agent.run(prompt)

print(response)




> Entering new AgentExecutor chain...
 I should use the GetCurrentWeather tool to get the current weather for Seoul.
Action: GetCurrentWeather
Action Input: 서울, unit(optional)
Observation: {"장소": "서울", "온도": "24", "단위": " unit(optional)", "예보": ["비"]}
Thought: I now know the final answer
Final Answer: 서울의 현재 날씨는 비가 오며 온도는 24도입니다.

> Finished chain.
서울의 현재 날씨는 비가 오며 온도는 24도입니다.


In [8]:
async for chunk in agent.astream(
    {"input": "서울의 날씨를 알려줘."}
):
    # Agent Action
    if "actions" in chunk:
        for action in chunk["actions"]:
            print(f"Calling Tool: `{action.tool}` with input `{action.tool_input}`")
    # Observation
    elif "steps" in chunk:
        for step in chunk["steps"]:
            print(f"Tool Result: `{step.observation}`")
    # Final result
    elif "output" in chunk:
        print(f'Final Output: {chunk["output"]}')
    else:
        raise ValueError()
    print("---")



> Entering new None chain...
Calling Tool: `GetCurrentWeather` with input `서울, unit(optional)`
---
 I should use the GetCurrentWeather tool to get the current weather for Seoul.
Action: GetCurrentWeather
Action Input: 서울, unit(optional)
Observation: {"장소": "서울", "온도": "24", "단위": " unit(optional)", "예보": ["비"]}
Thought:Tool Result: `{"장소": "서울", "온도": "24", "단위": " unit(optional)", "예보": ["비"]}`
---
 I now know the final answer
Final Answer: 서울의 현재 날씨는 비가 오며 온도는 24도입니다.

> Finished chain.
Final Output: 서울의 현재 날씨는 비가 오며 온도는 24도입니다.
---


In [9]:
import json
from langchain.tools import tool

@tool
async def get_current_weather_openai(location:str, unit:str="섭씨") -> str:
    """Return weather info for given location"""
    weather_info = {
        "장소": location,
        "온도": "24",
        "단위": unit,
        "예보": ["비"],
    }
    return json.dumps(weather_info, ensure_ascii=False)


In [10]:
weather = await get_current_weather_openai.ainvoke({"location":"서울"})
weather

'{"장소": "서울", "온도": "24", "단위": "섭씨", "예보": ["비"]}'

In [11]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.tools import tool
from langchain_core.callbacks import Callbacks
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

model = ChatOpenAI(temperature=0, streaming=True)
prompt = hub.pull("hwchase17/openai-tools-agent")
tools = [get_current_weather_openai]
agent = create_openai_tools_agent(
    model.with_config({"tags": ["agent_llm"]}), tools, prompt
)
agent_executor = AgentExecutor(agent=agent, tools=tools).with_config(
    {"run_name": "Agent"}
)

/tmp/ipykernel_21732/4137822799.py:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(temperature=0, streaming=True)
/home/ubuntu/.local/lib/python3.10/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [12]:
import pprint

chunks = []

async for chunk in agent_executor.astream(
    {"input": "서울의 날씨는 어떤지 알려줘."}
):
    chunks.append(chunk)
    print("------")
    pprint.pprint(chunk, depth=1)

------
{'actions': [...], 'messages': [...]}
------
{'messages': [...], 'steps': [...]}
------
{'messages': [...], 'output': '서울의 현재 날씨는 24°C이고 비가 오고 있습니다.'}


In [13]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt

/home/ubuntu/.local/lib/python3.10/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [14]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

prompt.messages[0] = SystemMessagePromptTemplate.from_template("당신은 친절한 어시스턴트입니다.")

In [15]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='